<a href="https://colab.research.google.com/github/djrl10/trying/blob/master/lstm_okay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [65]:
import tensorflow as tf
import pandas as pd
from io import StringIO
from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from keras.models import Sequential

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import codecs
import os
import glob
import re , itertools
from html import unescape
!pip install unidecode
from unidecode import unidecode
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.activations import elu, softmax
from sklearn.metrics import confusion_matrix
import seaborn
from keras.layers.core import Activation
from keras.models import Sequential,load_model
from keras.layers import LSTM
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [66]:
print(tf.__version__)

2.5.0


In [67]:
df = pd.read_csv("/content/drive/My Drive/datadataset/KC1.csv")

In [68]:
df.columns

Index(['loc', 'v(g)', 'ev(g)', 'iv(g)', 'n', 'v', 'l', 'd', 'i', 'e', 'b', 't',
       'lOCode', 'lOComment', 'lOBlank', 'locCodeAndComment', 'uniq_Op',
       'uniq_Opnd', 'total_Op', 'total_Opnd', 'branchCount', 'class'],
      dtype='object')

In [69]:
df.dtypes

loc                  float64
v(g)                 float64
ev(g)                float64
iv(g)                float64
n                    float64
v                    float64
l                    float64
d                    float64
i                    float64
e                    float64
b                    float64
t                    float64
lOCode                 int64
lOComment              int64
lOBlank                int64
locCodeAndComment      int64
uniq_Op              float64
uniq_Opnd            float64
total_Op             float64
total_Opnd           float64
branchCount          float64
class                   bool
dtype: object

In [70]:
df.shape

(2109, 22)

In [71]:
df.isnull().sum()

loc                  0
v(g)                 0
ev(g)                0
iv(g)                0
n                    0
v                    0
l                    0
d                    0
i                    0
e                    0
b                    0
t                    0
lOCode               0
lOComment            0
lOBlank              0
locCodeAndComment    0
uniq_Op              0
uniq_Opnd            0
total_Op             0
total_Opnd           0
branchCount          0
class                0
dtype: int64

In [72]:
df.head()

,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,b,t,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,class
0,1.1,1.4,1.4,1.4,1.3,1.30,1.30,1.30,1.30,1.30,1.30,1.30,2,2,2,2,1.2,1.2,1.2,1.2,1.4,False
1,1.0,1.0,1.0,1.0,1.0,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1,1,1,1,1.0,1.0,1.0,1.0,1.0,True
2,83.0,11.0,1.0,11.0,171.0,927.89,0.04,23.04,40.27,21378.61,0.31,1187.70,65,10,6,0,18.0,25.0,107.0,64.0,21.0,True
3,46.0,8.0,6.0,8.0,141.0,769.78,0.07,14.86,51.81,11436.73,0.26,635.37,37,2,5,0,16.0,28.0,89.0,52.0,15.0,True
4,25.0,3.0,1.0,3.0,58.0,254.75,0.11,9.35,27.25,2381.95,0.08,132.33,21,0,2,0,11.0,10.0,41.0,17.0,5.0,True


In [73]:
# One-hot encoding
classification = pd.get_dummies(df[df.columns[1]])
print(classification.shape)

(2109, 31)


In [74]:
X =df.drop([df.columns[0], df.columns[1], df.columns[21]], axis=1)   # drop the target variable from df 

In [75]:
X.head() 

,ev(g),iv(g),n,v,l,d,i,e,b,t,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount
0,1.4,1.4,1.3,1.30,1.30,1.30,1.30,1.30,1.30,1.30,2,2,2,2,1.2,1.2,1.2,1.2,1.4
1,1.0,1.0,1.0,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1,1,1,1,1.0,1.0,1.0,1.0,1.0
2,1.0,11.0,171.0,927.89,0.04,23.04,40.27,21378.61,0.31,1187.70,65,10,6,0,18.0,25.0,107.0,64.0,21.0
3,6.0,8.0,141.0,769.78,0.07,14.86,51.81,11436.73,0.26,635.37,37,2,5,0,16.0,28.0,89.0,52.0,15.0
4,1.0,3.0,58.0,254.75,0.11,9.35,27.25,2381.95,0.08,132.33,21,0,2,0,11.0,10.0,41.0,17.0,5.0


In [76]:
y = df[[df.columns[21]]]

In [77]:
y.head()

,class
0,False
1,True
2,True
3,True
4,True


In [78]:
# Splitting the dataset into the Training set and Test set
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=1)

In [79]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train1 = sc.fit_transform(X_train)
X_test1 = sc.transform(X_test)

In [80]:
#Size of train test data
X_train1.shape , y_train.shape

((1476, 19), (1476, 1))

In [81]:
X_test1.shape , y_test.shape

((633, 19), (633, 1))

In [82]:
y_test

,class
996,False
1413,False
1651,False
845,False
1690,False
...,...
1011,False
340,False
1737,False
1943,False


In [83]:
y_train

,class
2033,False
643,False
1708,False
531,False
697,False
...,...
960,False
905,False
1096,False
235,True


In [84]:
X_test1

array([[-0.29821187, -0.44178155, -0.5395649 , ..., -0.53944936,
        -0.53568284, -0.45410204],
       [ 7.72091053,  4.45666421,  4.14499159, ...,  3.99474693,
         4.35830539,  5.22497548],
       [-0.29821187, -0.44178155, -0.50477859, ..., -0.48347163,
        -0.53568284, -0.45410204],
       ...,
       [-0.29821187, -0.44178155, -0.56275577, ..., -0.55810861,
        -0.56608029, -0.45410204],
       [-0.29821187, -0.44178155, -0.52796946, ..., -0.52079012,
        -0.53568284, -0.45410204],
       [-0.29821187, -0.44178155, -0.46999228, ..., -0.46481239,
        -0.47488796, -0.45410204]])

In [85]:
X_train1

array([[-0.29821187,  0.13450618, -0.08734286, ..., -0.05430902,
        -0.14051609,  0.03973079],
       [-0.29821187,  0.13450618,  0.16775675, ...,  0.15094267,
         0.19385578,  0.03973079],
       [-0.29821187, -0.44178155, -0.37722879, ..., -0.39017541,
        -0.35329819, -0.20718562],
       ...,
       [-0.29821187,  0.13450618, -0.57435121, ..., -0.57676785,
        -0.56608029,  0.03973079],
       [-0.29821187,  0.13450618, -0.21489267, ..., -0.22224222,
        -0.20131098,  0.03973079],
       [ 4.15685613,  2.15151326,  0.82869665, ...,  0.80401621,
         0.86259951,  2.0150621 ]])

#build LSTM Model

In [86]:
def build_model(layers):
    d = 0.3
    model = Sequential()

    model.add(LSTM(256), return_sequences=True)
    model.add(Dropout(d))

    model.add(LSTM(256), return_sequences=False)
    model.add(Dropout(d))

    model.add(Dense(32,kernel_initializer="uniform",activation='relu'))
    model.add(Dense(1,kernel_initializer="uniform",activation='linear'))

    # adam = keras.optimizers.Adam(decay=0.2)

    history=model.compile(loss='mse',optimizer='adam', metrics=['accuracy'])
    
    return model

In [87]:
model = build_model(X_train1.shape)

TypeError: ignored

In [ ]:
model.summary()

In [ ]:
print('Train...')
model.fit(X_train1, y_train, epochs=50)
pred = model.predict(X_test1, verbose=0)
predict_classes = model.predict_classes(X_test1, verbose=0)
yhat_probs = pred[:, 0]
yhat_classes = predict_classes[:, 0]

In [ ]:
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test, yhat_classes)
print('Accuracy: %f' % accuracy)  
# precision tp / (tp + fp)
precision = precision_score(y_test, yhat_classes)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test, yhat_classes)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test, yhat_classes)
print('F1 score: %f' % f1)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper right')
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='lower right')
plt.show()

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
svc_model = SVC()
svc_model.fit(X_train,y_train)
svc_pred = svc_model.predict(X_test)

In [ ]:
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(svc_pred, y_test)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(svc_pred, y_test)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(svc_pred, y_test)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(svc_pred, y_test)
print('F1 score: %f' % f1)